In [1]:
#import necessary packages
import pandas as pd
from datetime import datetime, timedelta, timezone
import os
from PyEIS import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import csv
import glob
import statistics
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import math
from colour import Color
import matplotlib.ticker as ticker
import re

from matplotlib.pyplot import rc
rc('text',usetex=True)
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']},size='16')
rc('text.latex', preamble=r'\usepackage{sfmath}')

In [3]:
import re
import numpy as np

# Define a function to calculate the Euclidean distance between two points
def calculate_distance(point1, point2):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2))**2))

# Define a function to modify the file
def modify_file(input_file_path, output_file_path):
    # Read the file
    with open(input_file_path, 'r') as file:
        lines = file.readlines()

    # Initialize the new lines
    new_lines = []

    # Initialize the P and S sites
    p_sites = []
    s_sites = []

    # Iterate over the lines
    for line in lines:
        # If the line contains 'site P1'
        if 'site P1' in line:
            # Extract the coordinates
            x = float(re.search('x  (\\d+\\.\\d+)', line).group(1))
            y = float(re.search('y  (\\d+\\.\\d+)', line).group(1))
            z = float(re.search('z  (\\d+\\.\\d+)', line).group(1))

            # Add the P site to the list
            p_sites.append((x, y, z))

            # Replace 'site P1' with 'site PX'
            line = line.replace('site P1', f'site P{len(p_sites)}')

            # Replace the number following 'beq' with 'beq_P'
            line = re.sub(r'beq\s+\d+(\.\d+)?', 'Beqpdfgui(,beq_P)', line)

            # Add the line to the new lines
            new_lines.append(line)

        # If the line contains 'site S2'
        elif 'site S2' in line:
            # Extract the coordinates
            x = float(re.search('x  (\\d+\\.\\d+)', line).group(1))
            y = float(re.search('y  (\\d+\\.\\d+)', line).group(1))
            z = float(re.search('z  (\\d+\\.\\d+)', line).group(1))

            # Add the S site to the list
            s_sites.append((x, y, z, line))

        # If the line does not contain 'site P1' or 'site S2', add it to the new lines
        elif 'site P1' not in line and 'site S2' not in line:
            # If the line contains 'site Li'
            if 'site Li' in line:
                # Replace the number following 'beq' with 'beq_Li'
                line = re.sub(r'beq\s+\d+(\.\d+)?', 'Beqpdfgui(,beq_Li)', line)
                if 'site Li1'in line:
                    line=re.sub(r'occ Li\s+\d+(\.\d+)?','occ Li =Li_48h_occ;',line)
                if 'site Li2'in line:
                    line=re.sub(r'occ Li\s+\d+(\.\d+)?','occ Li =2*(0.5-Li_48h_occ);',line)
            # If the line contains 'site Cl1', 'site Br1', or 'site I1'
            elif 'site Cl1' in line or 'site Br1' in line or 'site I1' in line:
                # Replace the number following 'beq' with 'beq_X1'
                line = re.sub(r'beq\s+\d+(\.\d+)?', 'Beqpdfgui(,beq_X1)', line)
                # Replace occupancy with appropriate value--CHECK TO MAKE SURE THE SITE IS CORRECT
                # IN SHIFTED CELL, 4a site will not have 0 in XY or Z
                # IN SHIFTED CELL, some 4d sites WILL have 0 in X,Y,or Z. 
                # MAKE SURE ASSIGNMENT IS CORRECT and that S identifiers and X identifiers match
                #in this case Cl1 is 4a site
                line = re.sub(r'occ Br\s+\d+(\.\d+)?', 'occ Br =X_4a_occ;', line)

            # If the line contains 'site Cl2', 'site Br2', or 'site I2'
            elif 'site Cl2' in line or 'site Br2' in line or 'site I2' in line:
                # Replace the number following 'beq' with 'beq_X2'
                line = re.sub(r'beq\s+\d+(\.\d+)?', 'Beqpdfgui(,beq_X2)', line)
                # Replace occupancy with appropriate value--CHECK TO MAKE SURE THE SITE IS CORRECT
                # in this case Cl2 is the 4d site
                line = re.sub(r'occ Br\s+\d+(\.\d+)?', 'occ Br =X_4d_occ;', line)
                
            # If the line contains 'site S1'
            elif 'site S1' in line:
                # Replace the number following 'beq' with 'beq_S1'
                line = re.sub(r'beq\s+\d+(\.\d+)?', 'Beqpdfgui(,beq_S1)', line)
                # Replace occupancy with appropriate value
                #in this case S1 is the 4d site
                #S occupancy on 4d site will be same as X occupancy on 4a
                line = re.sub(r'occ S\s+\d+(\.\d+)?', 'occ S =S_4d_occ;', line)
                

            # If the line contains 'site S3'
            elif 'site S3' in line:
                # Replace the number following 'beq' with 'beq_S3'
                line = re.sub(r'beq\s+\d+(\.\d+)?', 'Beqpdfgui(,beq_S3)', line)
                # Replace occupancy with appropriate value
                #in this case S1 is the 4a site
                #S occupancy on 4a site will be same as X occupancy on 4d
                line = re.sub(r'occ S\s+\d+(\.\d+)?', 'occ S =S_4a_occ;', line)
            # Add the line to the new lines
            new_lines.append(line)

    # Initialize the renamed S sites
    renamed_s_sites = []

    # Initialize the rigid bodies
    rigid_bodies = []

    # Iterate over the P sites
    for i, p_site in enumerate(p_sites, start=1):
        # Calculate the distances to the S sites
        distances = [calculate_distance(p_site, s_site[:3]) for s_site in s_sites]

        # Get the indices of the 4 closest S sites
        indices = np.argsort(distances)[:4]

        # Rename the 4 closest S sites
        for j, index in enumerate(indices, start=1):
            # Get the original line
            line = s_sites[index][3]

            # Replace 'site S2' with 'site SXY'
            line = line.replace('site S2', f'site S{i}{j}')
            
            # Replace the number following 'beq' with 'beq_Sbound'
            line = re.sub(r'beq\s+\d+(\.\d+)?', 'Beqpdfgui(,beq_Sbound)', line)


            # Add the line to the renamed S sites
            renamed_s_sites.append(line)

        # Add the rigid bodies to the list
        rigid_bodies.append(f"""2\tc""")

    # Add the renamed S sites and the rigid bodies to the new lines
    new_lines = new_lines + ["\n"] + renamed_s_sites + ["\n", "prm r 2.048 min = 2; max = 2.1;\n", "\n"] + rigid_bodies

    # Write the new lines to the output file
    with open(output_file_path, 'w') as file:
        file.writelines(new_lines)


In [5]:
#name of the .str file we are targetting with this 
target_str='Li6PS5Br_P1_3by3'

output_name='Consolidated strs/3by3/Li6PS5Br_3by3_P1_Anionsitedisorder_dualLi_limited rotation_beqpdfgui_site disorder_Lioccref'

#filepath for the folder in which the string is stored
target_folder='C:/Users/austi/OneDrive - Colorado School of Mines/Research/Beamline Data/CLS_12_2023/Str/'

structure_path=''.join((target_folder,target_str,'.str'))

output_folder='C:/Users/austi/OneDrive - Colorado School of Mines/Research/Beamline Data/CLS_12_2023/Python Processed Str/'

output_path=''.join((output_folder,output_name,'.txt'))

In [7]:

modify_file(structure_path,output_path)